In [1]:
# globally useful imports of standard libraries needed in this notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os.path
from datetime import datetime

# specific libraries or classes needed for the work in this notebook
#from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import LogisticRegressionCV
#from sklearn.svm import SVC
#from sklearn.naive_bayes import GaussianNB
#from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif, chi2, SelectFpr, f_regression, mutual_info_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import ExtraTreesClassifier
#from sklearn.pipeline import Pipeline
# need to use Pipeline from imblearn to add in a downsample or upsample
# to cross validation training
from imblearn.pipeline import Pipeline

# import project specific modules used in this notebook
import sys
sys.path.append('../src')
import mindwandering.data
import mindwandering.features
import mindwandering.evaluate
from mindwandering.data import FeatureScalerTransformer
from mindwandering.data import WinsorizationOutlierTransformer
from mindwandering.data import VIFThresholdTransformer
from mindwandering.data import FeatureSelectionTransformer
from mindwandering.data import ClassImbalanceTransformer
from mindwandering.data import GridSearchProgressHack

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
from tensorflow.keras.losses import binary_crossentropy

from kerastuner.tuners import Hyperband,BayesianOptimization
import kerastuner

In [3]:
# Load Data and Features
df_features = mindwandering.data.get_df_features()
mind_wandered_label = mindwandering.data.get_mind_wandered_label()
participant_ids = mindwandering.data.get_participant_ids()

In [4]:
inputs_shape=df_features.shape[1]

In [5]:
import tensorflow
auc_roc = tensorflow.keras.metrics.AUC(name='auc_roc',num_thresholds=20000,curve='ROC')
#auc_roc = tensorflow.keras.metrics.AUC(name='auc_roc',curve='ROC')

In [6]:
from sklearn.model_selection import train_test_split
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(df_features, mind_wandered_label, test_size=0.33,random_state=123,stratify=mind_wandered_label)

In [7]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch


"""def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu'))
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu'))
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy',auc_roc])
    return model"""

"def build_model(hp):\n    model = keras.Sequential()\n    model.add(layers.Dense(units=hp.Int('units',\n                                        min_value=32,\n                                        max_value=512,\n                                        step=32),\n                           activation='relu'))\n    model.add(layers.Dense(units=hp.Int('units',\n                                        min_value=32,\n                                        max_value=512,\n                                        step=32),\n                           activation='relu'))\n    model.add(layers.Dense(units=hp.Int('units',\n                                        min_value=32,\n                                        max_value=512,\n                                        step=32),\n                           activation='relu'))\n    model.add(layers.Dense(1, activation='sigmoid'))\n    model.compile(\n        optimizer=keras.optimizers.Adam(\n            hp.Choice('learning_rate',\n         

In [8]:
def build_model(hp):
    if hp:
        dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5)
        num_units = hp.Choice('num_units', values=[8, 16, 32])
        learning_rate = hp.Float('learning_rate', min_value=0.0001, max_value=0.1)
        num_hidden_layers = hp.Choice('num_hidden_layers', values=[1, 2, 3])
    else:
        dropout_rate = 0.1
        num_units = 8
        learning_rate = 1e-4
        num_hidden_layers = 1
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(num_units, activation='relu',input_shape=(inputs_shape,)))
    
    for _ in range(0, num_hidden_layers):
        model.add(tf.keras.layers.Dense(num_units, activation='relu'))
        model.add(tf.keras.layers.Dropout(dropout_rate))
    
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        metrics=['accuracy',auc_roc]
    )
    
    return model

In [9]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir1',
    project_name='Keras-tuner-May-4-2021')

Traceback (most recent call last):
  File "/Users/subrotosingha/opt/anaconda3/lib/python3.8/site-packages/kerastuner/engine/hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-8-db3f69f4d0f1>", line 13, in build_model
    model = tf.keras.models.Sequential()
NameError: name 'tf' is not defined
Traceback (most recent call last):
  File "/Users/subrotosingha/opt/anaconda3/lib/python3.8/site-packages/kerastuner/engine/hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-8-db3f69f4d0f1>", line 13, in build_model
    model = tf.keras.models.Sequential()
NameError: name 'tf' is not defined
Traceback (most recent call last):
  File "/Users/subrotosingha/opt/anaconda3/lib/python3.8/site-packages/kerastuner/engine/hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-8-db3f69f4d0f1>", line 13, in build_model
    model = tf.keras.models.Sequential()
NameError: name 'tf

Invalid model 0/5
Invalid model 1/5
Invalid model 2/5


Traceback (most recent call last):
  File "/Users/subrotosingha/opt/anaconda3/lib/python3.8/site-packages/kerastuner/engine/hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-8-db3f69f4d0f1>", line 13, in build_model
    model = tf.keras.models.Sequential()
NameError: name 'tf' is not defined
Traceback (most recent call last):
  File "/Users/subrotosingha/opt/anaconda3/lib/python3.8/site-packages/kerastuner/engine/hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-8-db3f69f4d0f1>", line 13, in build_model
    model = tf.keras.models.Sequential()
NameError: name 'tf' is not defined
Traceback (most recent call last):
  File "/Users/subrotosingha/opt/anaconda3/lib/python3.8/site-packages/kerastuner/engine/hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-8-db3f69f4d0f1>", line 13, in build_model
    model = tf.keras.models.Sequential()
NameError: name 'tf

Invalid model 3/5
Invalid model 4/5
Invalid model 5/5


RuntimeError: Too many failed attempts to build model.

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

In [ ]:
tuner.results_summary()

In [ ]:

tuner = Hyperband(
    build_model,
    optimizer=keras.optimizers.Adam(0.01),
    loss='binary_crossentropy',
    metrics=[tensorflow.keras.metrics.AUC(name='auc_roc',num_thresholds=20000,curve='ROC')],
    objective=kerastuner.Objective('auc_roc', direction='max'),
    max_epochs=40,
    directory='my_dir_new2',
    project_name='Keras-tuner-May-4-2021-9:39pm')

#tuner.search(x, y,
             #validation_data=(val_x, val_y))

tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

In [ ]:
tuner.results_summary()

In [ ]:
tuner.search_space_summary()